### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 

#### Questions:

a) Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

b) Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 

Please perform all the necessary computations "by hands" rather than using python code.

In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
import statsmodels.formula.api as smf
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
#import shapefile

Populating the interactive namespace from numpy and matplotlib


In [77]:
y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

In [78]:
data

,Stolen?,Color,Type,Origin
0,1,1,1,1
1,0,1,1,1
2,1,1,1,1
3,0,2,1,1
4,1,2,1,2
5,0,2,2,2
6,1,2,2,2
7,0,2,2,1
8,0,1,2,2
9,1,1,1,2


# Answer
### (a)

P(Red|Stolen) = 3/5

P(SUV|Stolen) = 1/5

P(Domestic|Stolen) = 2/5

P(Red|Not Stolen) = 2/5

P(SUV|Not Stolen) = 3/5

P(Domestic|Not Stolen) = 3/5


### (b)

p(stolen) = 3/5 * 1/5 * 2/5 * 1/2 = 3/125

p(not stolen) = 2/5 * 3/5 * 3/5 * 1/2 = 9/125

p(stolen) < p(not stolen)

#### So it won't be stolen.


### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:

a)Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

b)Perform the classification for the test sample. 

c)Compare your result to y_test and report the classification accuracy.

# Answer

In [125]:
data_train=pd.read_csv("dia_train.csv") 
y_train=data_train.iloc[:,1] 
X_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("dia_test.csv")
y_test=data_test.iloc[:,1]
X_test=data_test.iloc[:,2:]

In [126]:
train = pd.concat([y_train,X_train],axis=1)
test = pd.concat([y_test,X_test],axis=1)
test.head()

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
0,0,1,89,66,23,94,28.1,0.167,21
1,1,0,137,40,35,168,43.1,2.288,33
2,1,2,197,70,45,543,30.5,0.158,53
3,1,1,189,60,23,846,30.1,0.398,59
4,0,1,103,30,38,83,43.3,0.183,33


In [127]:
### (a)

def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

dp=trainNaiveBayes(train)

dp

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
mu1,0.699153,2.69091,111.467,69.2061,27.2,127.006,31.7091,0.468685,28.3939
sigma1,NaN,2.61006,24.6919,11.7133,10.4369,91.4861,6.33761,0.29175,8.53736
mu2,0.300847,4.07042,144.141,74.5634,33.4789,209.211,35.2239,0.639042,35.7887
sigma2,NaN,3.51862,30.6265,13.7993,9.7627,126.921,6.25849,0.439042,10.2635


In [128]:
### (b)

def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

C=classifyNaiveBayes(test,dp)

In [129]:
### (c)

#classification accuracy (over test set)
100.0*sum(C==y_test)/len(C)

79.11392405063292

In [130]:
#also report accuracy of the classifier over the training - it is slightly higher, although not that higher
Ct=classifyNaiveBayes(train,dp)
100.0*sum(Ct==y_train)/len(Ct)

76.27118644067797

### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.

#### Questions:

a) Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

b) Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook). 

In [3]:
data_train=pd.read_csv("EM_train.csv")
y_Label_train=data_train.iloc[:,1] 
X_Label_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("EM_test.csv")
y_Label_test=data_test.iloc[:,1]
X_Label_test=data_test.iloc[:,2:]

data_Unlabel=pd.read_csv("EM_Unlabel.csv")
X_Unlabel=data_Unlabel.iloc[:,1:]

In [4]:
train = pd.concat([y_Label_train,X_Label_train],axis=1)
test = pd.concat([y_Label_test,X_Label_test],axis=1)

In [6]:
### (a)

def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp

dp=trainNaiveBayesDiscrete(train)
dp

[          y    0    1    2
 1  0.555556    0  0.2    0
 2       NaN  0.2    0  0.2
 3       NaN  0.2    0    0
 4       NaN  0.2  0.2  0.2
 5       NaN    0    0    0
 6       NaN  0.4  0.6  0.6,           y     0     1     2
 1  0.444444   0.5  0.25  0.25
 2       NaN  0.25   0.5   0.5
 3       NaN     0  0.25     0
 4       NaN     0     0     0
 5       NaN  0.25     0     0
 6       NaN     0     0  0.25]

In [8]:
dp[0]

,y,0,1,2
1,0.555556,0,0.2,0
2,NaN,0.2,0,0.2
3,NaN,0.2,0,0
4,NaN,0.2,0.2,0.2
5,NaN,0,0,0
6,NaN,0.4,0.6,0.6


In [11]:
def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0 #initialize the empty array 
  for i in classData.index: #for al records to classify
    #start with the priors
    P1=dp[0][classData.columns[0]][1]; 
    P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
  return Y

C=classifyNaiveBayesDiscrete(test,dp)

acc=format(100.0*sum(C==y_Label_test)/len(y_Label_test))
print "We correctly classified {0} percents of the trips based on the labeled data only".format(acc)     

We correctly classified 63.8888888889 percents of the trips based on the labeled data only


In [153]:
### (b)

#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if (sum(sum((dp1[0]-dp[0])**2))+sum(sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp

#perform EM estimation for theta
dpEM=EM(X_Label_train,y_Label_train,X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(test,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_Label_test)/len(y_Label_test)
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -590.452072842
Iteration 2: log maximum liklihood = -572.346722303
Iteration 3: log maximum liklihood = -559.190769042
Iteration 4: log maximum liklihood = -556.633109435
Iteration 5: log maximum liklihood = -556.067798435
After EM we correctly classified 94.4444444444 percents of the trips


### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:

#### Question: 

a) Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 

b) Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

In [196]:
data=pd.read_csv("EM_Cluster.csv")
X=data.iloc[:,1:]

In [197]:
X.head()

,0,1,2
0,5,6,6
1,4,6,6
2,1,5,5
3,1,3,4
4,5,6,2


In [201]:
### (a)

# np.random.seed(2016)
#initialize theta randomly
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
for j in range(1,len(dp[0].T)):
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b
    
dpEM=EM([],[],X,dp)

dpEM

Iteration 1: log maximum liklihood = -550.154324849
Iteration 2: log maximum liklihood = -546.993947751
Iteration 3: log maximum liklihood = -543.900022896
Iteration 4: log maximum liklihood = -540.926855276
Iteration 5: log maximum liklihood = -538.942993536
Iteration 6: log maximum liklihood = -537.948294685
Iteration 7: log maximum liklihood = -537.307000599
Iteration 8: log maximum liklihood = -536.627131653
Iteration 9: log maximum liklihood = -535.699541814
Iteration 10: log maximum liklihood = -534.351403321
Iteration 11: log maximum liklihood = -532.420831855
Iteration 12: log maximum liklihood = -529.641212864
Iteration 13: log maximum liklihood = -525.370333094
Iteration 14: log maximum liklihood = -519.393160528
Iteration 15: log maximum liklihood = -512.961265175
Iteration 16: log maximum liklihood = -508.411538536
Iteration 17: log maximum liklihood = -506.972542264


[          y            0            1           2
 1  0.499138      0.40239     0.253141    0.381396
 2       NaN     0.179784        0.467     0.29745
 3       NaN      0.21471     0.193681     0.18712
 4       NaN     0.150241   0.00845112   0.0445784
 5       NaN    0.0528737    0.0776716   0.0850116
 6       NaN  6.24274e-09  5.50834e-05  0.00444371,
           y          0           1           2
 1  0.500862  0.0553506   0.0329525  0.00021356
 2       NaN  0.0490127  0.00997817   0.0838703
 3       NaN  0.0712514   0.0351639   0.0797316
 4       NaN   0.287617    0.276801    0.183753
 5       NaN    0.30859    0.245848    0.314593
 6       NaN   0.228178    0.399257    0.337839]

In [219]:
### (b)

estimate={}
for i in range(10):
    #initialize theta randomly
    dp[0].iloc[0,0]=np.random.uniform(0,1)
    dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
    for j in range(1,len(dp[0].T)):
        b=np.random.uniform(0,1,len(dp[0]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[0].iloc[:,j]=b     
        
    for j in range(1,len(dp[1].T)):
        b=np.random.uniform(0,1,len(dp[1]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[1].iloc[:,j]=b
            
    dpEM=EM([],[],X,dp)
    
    
    print "time",i+1
    print dpEM
    estimate[i]=dpEM

Iteration 1: log maximum liklihood = -553.265915479
Iteration 2: log maximum liklihood = -548.358768562
Iteration 3: log maximum liklihood = -544.629912759
Iteration 4: log maximum liklihood = -542.034504784
Iteration 5: log maximum liklihood = -540.497706012
Iteration 6: log maximum liklihood = -539.52502777
Iteration 7: log maximum liklihood = -538.859354936
Iteration 8: log maximum liklihood = -538.440146378
time 1
[         y          0          1          2
1  0.63895   0.256017   0.153824   0.288186
2      NaN   0.159325   0.343516   0.222033
3      NaN   0.114498  0.0674352  0.0193632
4      NaN   0.340716  0.0530326   0.116631
5      NaN  0.0806494   0.253348   0.254187
6      NaN  0.0487947   0.128843  0.0995997,          y           0            1          2
1  0.36105    0.180001     0.123448  0.0175598
2      NaN     0.03458    0.0515312   0.134629
3      NaN    0.193045     0.197197   0.335026
4      NaN  0.00373057      0.30182   0.110135
5      NaN    0.358459  7.67473e-

In [233]:
### estimate average label and its standard error for each record

estimate_mean = dp
estimate_mean[0].iloc[0,0]=0
estimate_mean[1].iloc[0,0]=0
for j in range(1,len(dp[0].T)):
    estimate_mean[0].iloc[:,j]=0
for j in range(1,len(dp[1].T)):
    estimate_mean[1].iloc[:,j]=0

for i in range(10):
    estimate_mean[0] += estimate[i][0]
    estimate_mean[1] += estimate[i][1]
    
estimate_mean[0] = estimate_mean[0]/10
estimate_mean[1] = estimate_mean[1]/10
print "average label for  each record"
estimate_mean

average label for  each record


[         y         0         1         2
 1  0.51419  0.214454  0.132476  0.181031
 2      NaN  0.111158  0.230297  0.182935
 3      NaN  0.126038  0.101209  0.111639
 4      NaN   0.24219  0.139864  0.123755
 5      NaN  0.185959  0.180481  0.219059
 6      NaN    0.1202  0.215673   0.18158,
          y         0         1         2
 1  0.48581  0.238837   0.15109  0.191339
 2      NaN  0.113189  0.234621  0.194611
 3      NaN  0.160726  0.130272  0.162803
 4      NaN  0.186899  0.153567  0.105403
 5      NaN  0.184997  0.137434   0.17675
 6      NaN  0.115352  0.193016  0.169094]

In [256]:
### standard error for each record

estimate_std = dp
estimate_std[0].iloc[0,0]=0
estimate_std[1].iloc[0,0]=0
for j in range(1,len(dp[0].T)):
    estimate_std[0].iloc[:,j]=0
for j in range(1,len(dp[1].T)):
    estimate_std[1].iloc[:,j]=0
    
for i in range(10):
    
    estimate_std[0] += (estimate[i][0]-estimate_mean[0])**2
    estimate_std[1] += (estimate[i][1]-estimate_mean[1])**2

estimate_std[0] = estimate_std[0]/10
estimate_std[1] = estimate_std[1]/10

estimate_std[0].iloc[0,0]=sqrt(estimate_std[0].iloc[0,0])
estimate_std[1].iloc[0,0]=sqrt(estimate_std[1].iloc[0,0])
for j in range(1,len(dp[0].T)):
    for i in range(len(dp[0])):
        estimate_std[0].iloc[i,j]=sqrt(estimate_std[0].iloc[i,j])
for j in range(1,len(dp[1].T)):
    for i in range(len(dp[1])):
        estimate_std[1].iloc[i,j]=sqrt(estimate_std[1].iloc[i,j])
    
print "standard error for  each record"
estimate_std

standard error for  each record


[          y         0         1         2
 1  0.213931  0.229372  0.142577  0.278157
 2       NaN  0.103107  0.289723  0.149744
 3       NaN  0.130511  0.108952  0.110342
 4       NaN  0.158897  0.120497   0.10126
 5       NaN  0.145064  0.130196  0.167301
 6       NaN  0.123226  0.216293  0.169227,
           y         0         1         2
 1  0.206994   0.19274  0.131999  0.196698
 2       NaN  0.101219  0.217888  0.143115
 3       NaN  0.129797  0.113025  0.130853
 4       NaN  0.162051  0.141497  0.098487
 5       NaN  0.174845  0.125543  0.195226
 6       NaN  0.144227  0.283087  0.209618]